In [3]:
import paramiko
import  numpy as np
import time
from time import sleep  

In [42]:
# setup connection
def setup():
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect('10.227.27.145', username='root', password='lightlab')
    shell = ssh.invoke_shell()
    shell.send('telnet localhost 6571\n')
    while not shell.recv_ready():
        time.sleep(3)
    out = shell.recv(9999)
    print(out.decode("ascii"))
    return shell

In [54]:
# send a command and return the controller's response
def write_and_read(shell, cmd):
    shell.send(cmd+"\n")
    count = 0
    nTimeout = 3
    while ( (count<nTimeout) and (not shell.recv_ready()) ):
        time.sleep(1)
        count += 1
    if (count<nTimeout):
        out = shell.recv(99).decode()
        out = out.splitlines()
        out = out[1:len(out):2]
        # print("Send:"+cmd+", Receieved:"+out)
        return out
    else:
        print("Error: Time out!")
        return

In [55]:
def bit(x):
    return (1<<x)

def genmask(h,l):
    return ((1<<(h+1))-1) - ((1<<(l))-1)

In [56]:
# eeprom
def eeprom_read(shell, slot, addr, size):
    cmd = "2:{}:0:{},{}?".format(slot,addr,size)
    lines = write_and_read(shell, cmd)
    ret = np.zeros(size, dtype=int)
    for i in range(size):
        ret[i] = int(lines[i])
    return ret 

def eeprom_write(shell, slot, addr, data):
    size = len(data)
    cmd = "2:{}:0:{},{}:".format(slot,addr,size)
    for i in range(size):
        cmd += "{},".format(data[i])
    line = write_and_read(shell, cmd)[0]
    print(line)

In [46]:
# monitor
## bp
def mon_bp_rst(shell):
    cmd = "3:99:0:0,0?"
    line = write_and_read(shell, cmd)[0]
    # print(line) 

def mon_bp_read(shell, hw_addr, addr):
    cmd = "3:0:0:{},{}?".format(hw_addr, addr)
    ret = int(write_and_read(shell, cmd)[0])
    return ret 

def mon_bp_write(shell, hw_addr, addr, data):
    cmd = "3:0:0:{},{}:{}".format(hw_addr, addr, data)
    line = write_and_read(shell, cmd)

def mon_bp_setup(shell, hw_addr, slot_opy):
    mon_bp_write(shell, hw_addr, 0x0A, 0xE8)
    mon_bp_write(shell, hw_addr, 0x06, 0xFF)
    mon_bp_write(shell, hw_addr, 0x16, 0xFF)
    mon_bp_write(shell, hw_addr, 0x03, 0xFF)
    mon_bp_write(shell, hw_addr, 0x13, 0xFF)
    mon_bp_write(shell, hw_addr, 0x04, 0xFF)
    mon_bp_write(shell, hw_addr, 0x14, 0xFF)
    mon_bp_write(shell, hw_addr, 0x02, (slot_opy&0xFF))
    mon_bp_write(shell, hw_addr, 0x12, ((slot_opy>>8)&0xFF))

## db
def mon_db_rst(shell, slot):
    cmd = "4:{}:99:0?".format(slot)
    line = write_and_read(shell, cmd)[0]
    # print(line) 

def mon_db_read(shell, slot, hw_addr, addr):
    cmd = "4:{}:0:{},{}?".format(slot, hw_addr, addr)
    ret = int(write_and_read(shell, cmd)[0])
    return ret 

def mon_db_write(shell, slot, hw_addr, addr, data):
    cmd = "4:{}:0:{},{}:{}".format(slot, hw_addr, addr, data)
    line = write_and_read(shell, cmd)

def mon_db_setup(shell, slot, hw_addr, ch_opy):
    mon_db_write(shell, slot, hw_addr, 0x0A, 0xAA)
    mon_db_write(shell, slot, hw_addr, 0x06, 0xFF)
    mon_db_write(shell, slot, hw_addr, 0x16, 0xFF)
    mon_db_write(shell, slot, hw_addr, 0x03, 0xFF)
    mon_db_write(shell, slot, hw_addr, 0x13, 0xFF)
    mon_db_write(shell, slot, hw_addr, 0x04, 0xFF)
    mon_db_write(shell, slot, hw_addr, 0x14, 0xFF)
    mon_db_write(shell, slot, hw_addr, 0x02, ch_opy)
    mon_db_write(shell, slot, hw_addr, 0x12, ch_opy)

In [51]:
# adc
def adc_read(shell, slot, ch, addr, size):
    cmd = "0:{}:{}:{},{}?".format(slot, ch, addr, size)
    line = write_and_read(shell, cmd)
    ret = np.zeros(size, dtype=int)
    for i in range(size):
        ret[i] = int(line[i])
    return ret 

def adc_read8(shell, slot, ch, addr):
    cmd = "0:{}:{}:{},1?".format(slot, ch, addr)
    ret = int(write_and_read(shell, cmd)[0])
    return ret 

def adc_read24(shell, slot, ch, addr):
    cmd = "0:{}:{}:{},3?".format(slot, ch, addr)
    line = write_and_read(shell, cmd)
    ret = (int(line[0])<<16)+(int(line[1])<<8)+int(line[2])
    return ret 

def adc_write(shell, slot, ch, addr, data):
    size = len(data)
    cmd = "0:{}:{}:{},{}:".format(slot, ch, addr, size)
    for i in range(size):
        cmd += "{},".format(data[i])
    line = write_and_read(shell, cmd)[0]
    print(line)

def adc_write8(shell, slot, ch, addr, data):
    cmd = "0:{}:{}:{},1:{}".format(slot, ch, addr, data)
    line = write_and_read(shell, cmd)[0]
    print(line)

def adc_write8_mask(shell, slot, ch, addr, mask, data):
    cmd = "0:{}:{}:{},1?".format(slot, ch, addr)
    reg_data = int(write_and_read(shell, cmd)[0])
    reg_data &= ~mask
    reg_data |= data
    cmd = "0:{}:{}:{},1:{}".format(slot, ch, addr, reg_data)
    line = write_and_read(shell, cmd)[0]
    print(line)

def adc_command(shell, slot, ch, cmd):
    size = len(data)
    cmd = "0:{}:{}:0,0:{}".format(slot, ch, cmd)
    line = write_and_read(shell, cmd)[0]
    print(line)

def adc_swreset(shell, slot, ch):
    adc_write8_mask(shell, slot, ch, 1, genmask(5,4), 0b11<<4)
    adc_command(shell, slot, ch, 0x90)

def adc_setup(shell, slot, ch):
    adc_swreset(shell, slot, ch)
    adc_write8(shell, slot, ch, 1, 0x26)
    adc_write8(shell, slot, ch, 2, 0x20)
    adc_write8(shell, slot, ch, 3, 0x10)
    adc_write8(shell, slot, ch, 4, 0xC0)
    adc_write8(shell, slot, ch, 8, 0x20)
    #delay 10

def adc_calibrate(shell, slot, ch):
    seq_data = adc_read8(shell, slot, ch, 8)
    if ( (seq_data&genmask(4,3)) != 0 ):
        print("SEQ ERROR!")
        return
    adc_write8_mask(shell, slot, ch, 1, genmask(7,6), 0)    
    adc_command(shell, slot, ch, 0xA0)
    #delau 200

def adc_prepare_convert(shell, slot, ch):
    adc_write8_mask(shell, slot, ch, 8, genmask(4,3), 0)
    adc_write8_mask(shell, slot, ch, 2, 0b1111, 0b1000)

def adc_convert(shell, slot, ch, channel):
    adc_write8_mask(shell, slot, ch, 8, genmask(7,5), (channel&0x7)<<5)
    adc_command(shell, slot, ch, 0xB7)
    #delay 1000
    data = adc_read24(shell, slot, ch, 14+channel)
    return data

In [50]:
# dac
def dac_read(shell, slot, ch, addr):
    cmd = "1:{}:{},0:{}?".format(slot, ch, addr)
    ret = int(write_and_read(shell, cmd)[0])
    return ret 

def dac_read32(shell, slot, ch, addr):
    cmd = "1:{}:{},1:{}?".format(slot, ch, addr)
    ret = int(write_and_read(shell, cmd)[0])
    return ret 

def dac_write(shell, slot, ch, addr, data):
    cmd = "1:{}:{}:{},0:{}".format(slot, ch, addr, data)
    line = write_and_read(shell, cmd)[0]
    print(line)

def dac_write_mask(shell, slot, ch, addr, mask, data):
    cmd = "1:{}:{}:{},1:{},{}".format(slot, ch, addr, mask, data)
    line = write_and_read(shell, cmd)[0]
    print(line)

def dac_setup(shell, slot, ch):
    dac_write(shell, slot, ch, 0x08, 0x15FA)
    dac_write(shell, slot, ch, 0x08, 0xAF51)
    #delay 500
    dac_write(shell, slot, ch, 0x08, 0xFCBA)
    #delay 1250
    while True:
        diag_result = dac_read(shell, slot, ch, 0x14)
        if (not(diag_results&bit(15))):
            break
    dac_write(shell, slot, ch, 0x14, (1<<13))
    dac_write_mask(shell, slot, ch, 0x09, genmask(11,10), bit(10))
    dac_write_mask(shell, slot, ch, 0x0C, 0b1001111, 0x1)
    while True:
        dcdc_status = dac_read(shell, slot, ch, 0x0C)
        if (not(dcdc_status&bit(12))):
            break
    dac_write_mask(shell, slot, ch, 0x0B, genmask(6,5), bit(6))
    while True:
        dcdc_status = dac_read(shell, slot, ch, 0x0C)
        if (not(dcdc_status&bit(12))):
            break
    dac_write(shell, slot, ch, 0x01, 0x0)
    dac_write_mask(shell, slot, ch, 0x06, 0b1101111, bit(5))
    while True:
        diag_result = dac_read(shell, slot, ch, 0x14)
        if (not(diag_result&bit(15))):
            break
    #delay 200
    dac_write(shell, slot, ch, 0x01, 0x0)
    dac_write_mask(shell, slot, ch, 0x0E, 0b10101, 0b10101)
    dac_write_mask(shell, slot, ch, 0x0D, 0b111111, 0b10101)
    #delay 1250

def dac_set_new_range(shell, slot, ch, range):
    dac_input_reg = dac_read(shell, slot, ch, 0x06)
    bipolar_range = ((dac_input_reg&0xF)==0x2) or ((dac_input_reg&0xF)==0x3) or ((dac_input_reg&0xF)==0xB) or ((dac_input_reg&0xF)==0xC)
    if (bipolar_range):
        dac_write(shell, slot, ch, 0x01, 0x8000)
    else:
        dac_write(shell, slot, ch, 0x01, 0x0000)
    dac_write_mask(shell, slot, ch, 0x06, 0b1101111, bit(5)|range)
    #delay 200
    while True:
        diag_result = dac_read(shell, slot, ch, 0x14)
        if (not(diag_result&bit(15))):
            break
    if (range&0x8):
        dc_dc_mode = 0x1
    else:
        dc_dc_mode = 0x2
    dac_write_mask(shell, slot, ch, 0x0B, genmask(6,5), dc_dc_mode<<5)
    while True:
        dcdc_status = dac_read(shell, slot, ch, 0x0C)
        if (not(dcdc_status&bit(12))):
            break

    bipolar_range = (range==0x2) or (range==0x3) or (range==0xB) or (range==0xC)
    if (bipolar_range):
        dac_write(shell, slot, ch, 0x01, 0x8000)
    else:
        dac_write(shell, slot, ch, 0x01, 0x0000)
    #delay 100

In [52]:
# Yun
def ping(shell):
    cmd = "5:0:0:0?"
    line = write_and_read(shell, cmd)[0]
    print(line)

# backplane
def bp_setup(shell, slot_opy):
    mon_bp_rst(shell)
    mon_bp_setup(shell, 0, slot_opy)
    mon_bp_setup(shell, 1, slot_opy)
    print("-----Backplane Setup-----")

# daughterboard
def db_setup(shell, slot, ch_opy):
    mon_db_rst(shell, slot)
    mon_db_setup(shell, slot, 0, ch_opy)
    print("-----DB ({}) Setup-----".format(slot))

def db_is_exist(shell, slot):
    mac_addr = eeprom_read(shell, slot, 250, 6)
    ret = (mac_addr[0]==128) and (mac_addr[1]==31) and (mac_addr[2]==18)
    return ret

def db_mon_diag(shell, slot):
    adc_fault = mon_db_read(shell, slot, 0, 0x07)
    power_fault = mon_db_read(shell, slot, 0, 0x17)
    return adc_fault, power_fault

# smu
def smu_is_exist(shell, slot, ch):
    chip_id = dac_read(shell, slot, ch, 0x17)
    ret = (chip_id!=0xFFFF) and (chip_id!=0x0000)
    return ret

def smu_reference_switch(shell, slot, ch, refres):
    dac_write_mask(shell, slot, ch, 0x0E, bit(2), refres)

def smu_output_enable(shell, slot, ch, enable):
    dac_write_mask(shell, slot, ch, 0x0E, bit(0), not enable)

def smu_setup(shell, slot, ch):
    dac_setup(shell, slot, ch)
    adc_setup(shell, slot, ch)
    adc_calibrate(shell, slot, ch)
    
    smu_reference_switch(shell, slot, ch, True)
    smu_output_enable(shell, slot, ch, True)

    dac_set_new_range(shell, slot, ch, 0)
    adc_prepare_convert(shell, slot, ch)

def smu_set_dac_code(shell, slot, ch, range, dac_code):
    dac_input_reg = dac_read(shell, slot, ch, 0x06)
    previous_range = DAC_CONFIG_RANGE_MODE&0xF
    if (range != previous_range):
        dac_set_new_range (shell, slot, ch, range)
    dac_write_mask(shell, slot, ch, 0x06, bit(6), bit(6))
    dac_write(shell, slot, ch, 0x01, dac_code)

def smu_measure_v_sense(shell, slot, ch, cal_info):
    adc_code_raw = adc_convert(shell, slot, ch, 2)
    adc_code = uint32_t( (float(cal_info[0])/0x800000*0.1+0.9)*int32_t(adc_code_raw)+cal_info[1]-0x800000);
    return adc_code